In [ ]:
# Importing required packages 
import pandas as pd
import matplotlib.pyplot as plt #(plotin histograms in cell #3)
import numpy as np
from scipy.stats import kde
from tabulate import tabulate
import scipy.stats as stats
import pandas as pd
from scipy.stats._continuous_distns import _distn_names
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import scipy.special as sse
from scipy.optimize import curve_fit
from functools import partial
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression,LassoLarsIC, LinearRegression, Ridge, Lasso
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Defining some constants 
nbins=30

In [ ]:
# Reading Data 
energydata = pd.read_csv("./energydata_complete.csv")
# Creating subsamples
# Choosing 10 continous colomns, RH_out, T_out and Appliance are the target variables
sample_energydata=energydata[[ 'T1','T2', 'T4', 'T5', 'RH9', 'RH_7', 'RH_3', 'RH_out', 'T_out','Appliances']] 
df_sample_colomn_energydata = pd.DataFrame(sample_energydata)